### Cover Set
To rapresent states is a good procedure to use sets instead of arrays and vectors because sets are unordered.

We need a set of the state we need and one for the states we don't need.

The problem we are trying to solve is to find a state (which is a set of sets of True and False which means)

In [10]:
from random import random
from functools import reduce
import numpy as np
from queue import PriorityQueue, LifoQueue, SimpleQueue
from threading import Thread
import time

PROBLEM_SIZE=50
NUM_SETS=4000
SETS=tuple(np.array([random() < .1 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))

def goal_check(state):
    return np.all(reduce(np.logical_or,[SETS[i] for i in state[0]],np.array([False for _ in range(PROBLEM_SIZE)])))

def get_true_row(state):
    return reduce(np.logical_or,[SETS[i] for i in state[0]],np.array([False for _ in range(PROBLEM_SIZE)]))

def get_distance_from_state(state, action): # Cost For Greedy Solution
    t=get_true_row(state)

    res=np.logical_and(SETS[action], np.logical_not(t))
    # if np.count_nonzero(t==True):
    #     print(f"State: {t}")
    #     print(f"Action: {SETS[action]}")
    #     print(f"Res: {res}")
    return np.count_nonzero(res==True)

def get_distance_from_goal(state):  #Cost (Euristhic) For Informed Solution
    t=get_true_row(state)
    return np.count_nonzero(t==False)
    


In the following code there are 4 possible solutions to achieve the goal. 
- Greedy finds the "best solution" in the frontieer
- Informed findes the "closest solution" to the solution while not considering the cost
- A* simple also consider a unit cost 
- A* complicated considers the best solution in the frontieer that is closer to the solution

In [11]:
def Solve( case):

    start=time.time()
    frontier=PriorityQueue()
    frontier.put((0,[set(), set(range(NUM_SETS))] , 0))

    t=frontier.get()
    current_state=t[1]
    counter=0
    name=""

    if case==0:
        name="Greedy"
    if case==1:
        name="Informed"
    if case==2:
        name="A* simple"
    if case==3:
        name="A* complicated"
    while not goal_check(current_state):
        counter+=1
        
        for action in current_state[1]:
            new_state=(current_state[0] | {action}, current_state[1] - {action})

            # --- Greedy Solution --- #
            if(case==0):
                d_to_next_state=t[2]- get_distance_from_state(current_state,action)
                frontier.put((d_to_next_state,new_state, d_to_next_state))

            # --- Informed Solution --- #
            if(case==1):
                frontier.put((get_distance_from_goal(new_state), new_state, t[2]))

            # --- A* easy Solution --- #
            if(case==2):
                frontier.put((get_distance_from_goal(new_state) + len(new_state[0]), new_state , t[2]))

            # --- A* Solution --- #
            if(case==3):
                d_to_next_state=t[2]- get_distance_from_state(current_state,action)
                frontier.put((get_distance_from_goal(new_state) + d_to_next_state, new_state, d_to_next_state))
            

        if goal_check(current_state):
            break

        t=frontier.get()
        current_state=t[1]
    
    end=time.time()-start
    
    print(f"Solved {name} in {counter} steps with {len(new_state[0])} tiles in {end} ms\n ")
    
state=[set(range(NUM_SETS)), set()]


The cost implemented for greedy solutions values as more expensive a step with lower number of "True" added to the state.

In [12]:
assert goal_check(state),"Not Solvable Problem"

Solve(0)
Solve(1)
Solve(2)
Solve(3)
# t1=Thread(target=Solve, args=[0])
# t2=Thread(target=Solve, args=[1])
# t3=Thread(target=Solve, args=[2])
# t4=Thread(target=Solve, args=[3])

# t1.start()
# t2.start()
# t3.start()
# t4.start()

# t1.join()
# t2.join()
# t3.join()
# t4.join()


Solved Greedy in 7 steps with 7 tiles in 4.001256942749023 ms
 
Solved Informed in 7 steps with 7 tiles in 3.9079980850219727 ms
 
Solved A* simple in 8 steps with 7 tiles in 5.185999870300293 ms
 
Solved A* complicated in 7 steps with 7 tiles in 4.501993417739868 ms
 


To confront the solutions I'm starting different thread that measure the time between the start and the end of the algorithm (with the same set of sets). \\ 
It is possible to notice that in this case A* simple usually solves the problem in 1 or 2 steps more than the others.